In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
import pandas as pd

# Importing data

In [3]:
raw_data = pd.read_csv("../data/train.csv")

In [4]:
clean_data = raw_data.drop(["Cabin", "Name", "PassengerId", "Ticket"], axis=1)

In [5]:
clean_data = pd.get_dummies(clean_data).fillna(-1)

In [6]:
train, test = train_test_split(clean_data)

In [7]:
def split_x_y(dataframe, target):
    return dataframe.drop(target, axis=1), dataframe[target]

In [8]:
X_train, y_train = split_x_y(train, "Survived")
X_test, y_test = split_x_y(test, "Survived")

# Baseline model

In [9]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [10]:
y_pred = rf.predict_proba(X_test)

In [11]:
auc = roc_auc_score(y_true=y_test, y_score=y_pred[:, 1])

In [12]:
auc

0.88508771929824559

# Deep Forest

## By Hand

In [13]:
from sklearn.model_selection import StratifiedKFold

In [14]:
rf1 = RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=4)
rf2 = RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=10)

In [15]:
rf1.fit(X_train, y_train)
y_pred_train_1 = rf1.predict_proba(X_train)
y_pred_test_1 = rf1.predict_proba(X_test)

y_pred_train_1 = pd.DataFrame(y_pred_train_1, columns=["rf1_0", "rf1_1"], index=X_train.index)
y_pred_test_1 = pd.DataFrame(y_pred_test_1, columns=["rf1_0", "rf1_1"], index=X_test.index)

In [16]:
rf2.fit(X_train, y_train)
y_pred_train_2 = rf2.predict_proba(X_train)
y_pred_test_2 = rf2.predict_proba(X_test)

y_pred_train_2 = pd.DataFrame(y_pred_train_2, columns=["rf2_0", "rf2_1"], index=X_train.index)
y_pred_test_2 = pd.DataFrame(y_pred_test_2, columns=["rf2_0", "rf2_1"], index=X_test.index)

In [17]:
hidden_train_1 = pd.concat([X_train, y_pred_train_1, y_pred_train_2], axis=1)
hidden_test_1 = pd.concat([X_test, y_pred_test_1, y_pred_test_2], axis=1)

In [18]:
hidden_train_1.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,rf1_0,rf1_1,rf2_0,rf2_1
161,2,40.0,0,0,15.750,1,0,0,0,1,0.152943,0.847057,0.005220,0.994780
748,1,19.0,1,0,53.100,0,1,0,0,1,0.642007,0.357993,0.775900,0.224100
185,1,-1.0,0,0,50.000,0,1,0,0,1,0.721587,0.278413,0.917578,0.082422
19,3,-1.0,0,0,7.225,1,0,1,0,0,0.316890,0.683110,0.076315,0.923685
250,3,-1.0,0,0,7.250,0,1,0,0,1,0.882603,0.117397,0.942527,0.057473


In [19]:
rf3 = RandomForestClassifier(n_estimators=300, n_jobs=-1)
rf3.fit(hidden_train_1, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=300, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [20]:
y_pred3 = rf3.predict_proba(hidden_test_1)

In [21]:
roc_auc_score(y_test, y_pred3[:, 1])

0.86537176274018379

## With API

In [22]:
from deepforest.layer import Layer

In [23]:
layer1 = Layer(RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=4), 
               RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=10))

In [24]:
layer1.fit(X_train, y_train)

In [26]:
layer1.predict(X_test)

AttributeError: 'Layer' object has no attribute 'predict'